In [ ]:
import torch as tn
try:
    import torchtt as tntt
except:
    print('Installing torchTT...')
    %pip install git+https://github.com/ion-g-ion/torchTT

In [ ]:
N = [10,11,12,13,14]
Rt = [1,3,4,5,6,1]
Rx = [1,6,6,6,6,1]
target = tntt.randn(N,Rt).round(0)
func = lambda x: 0.5*(x-target).norm(True)

In [ ]:
x0 = tntt.randn(N,Rx)
x =x0.clone()
for i in range(20):
    # compute riemannian gradient using AD    
    gr = tntt.manifold.riemannian_gradient(x,func)
    
    #stepsize length
    alpha = 1.0
    
    # update step
    x = (x-alpha*gr).round(0,Rx)    
    print('Value ' , func(x).numpy())

As a comparison, conventional gradient descent with respect to the TT cores is performed:

In [ ]:
y = x0.detach().clone()

for i in range(10000):
    tntt.grad.watch(y)
    fval = func(y)
    deriv = tntt.grad.grad(fval,y)    
    alpha = 0.00001 # for stability
    y = tntt.TT([y.cores[i].detach()-alpha*deriv[i] for i in range(len(deriv))])
    if (i+1)%500 == 0: print(func(y))
    

### Manifold tensor completion

One other task where the manifold learning can be applied is tensor completion.
The goal for this problem is to reconstruct a tensor in the TT format given only a few entries (possible noisy).